In [2]:
# import
import pandas as pd
import numpy as np
from pathlib import Path
import os

print("Current Working Directory:", os.getcwd())

BASE_DIR = Path(os.getcwd()).resolve().parent
RAW_DIR = BASE_DIR / "data" / "raw"

EDU_DIR = RAW_DIR / "education_data"

print("BASE_DIR:", BASE_DIR)
print("EDU_DIR:", EDU_DIR)

Current Working Directory: /Users/lyt/Desktop/is477-course_project-deepsick/notebooks
BASE_DIR: /Users/lyt/Desktop/is477-course_project-deepsick
EDU_DIR: /Users/lyt/Desktop/is477-course_project-deepsick/data/raw/education_data


In [3]:
# Inspect Education Data Directory Structure
print("=== Listing Education Data Directory ===")

year_folders = sorted([p for p in EDU_DIR.iterdir() if p.is_dir()])

for folder in year_folders:
    print(f"Found Year Folder: {folder.name}")
    files = list(folder.iterdir())
    
    if len(files) == 0:
        print("   (Empty folder — no files found)")
    else:
        for f in files:
            print("   -", f.name)

=== Listing Education Data Directory ===
Found Year Folder: 15-16
   - ccd_sea_059_1516_w_1a_011717_csv.zip
   - ccd_sea_052_1516_w_1a_011717_csv.zip
   - ccd_sea_052_1516_w_1a_011717_xls.zip
   - ccd_sea_059_1516_w_1a_011717_xlsx.zip
Found Year Folder: 17-18
   - ccd_sea_059_1718_l_1a_083118.zip
   - ccd_sea_052_1718_l_1a_083118.zip
Found Year Folder: 19-20
   - ccd_sea_052_1920_l_1a_082120.zip
   - ccd_sea_059_1920_l_1a_082120.zip
Found Year Folder: 21-22
   - ccd_sea_052_2122_l_1a_071722.zip
   - ccd_sea_059_2122_l_1a_071722.zip
Found Year Folder: 23-24
   - ccd_sea_052_2324_l_1a_073124.zip
   - ccd_sea_059_2324_l_1a_073124.zip


In [4]:
# Inspect ZIP Contents
import zipfile

print("=== Inspecting ZIP Contents ===")

for year_folder in sorted([f for f in EDU_DIR.iterdir() if f.is_dir()]):
    print(f"Year Folder: {year_folder.name}")
    
    for zfile in year_folder.iterdir():
        if zfile.suffix == ".zip":
            print(f" ZIP File: {zfile.name}")
            with zipfile.ZipFile(zfile, 'r') as z:
                for name in z.namelist():
                    print(f"     - {name}")

=== Inspecting ZIP Contents ===
Year Folder: 15-16
 ZIP File: ccd_sea_059_1516_w_1a_011717_csv.zip
     - ccd_sea_059_1516_w_1a_011717.csv
 ZIP File: ccd_sea_052_1516_w_1a_011717_csv.zip
     - ccd_sea_052_1516_w_1a_011717.csv
 ZIP File: ccd_sea_052_1516_w_1a_011717_xls.zip
     - ccd_sea_052_1516_w_1a_011717.xlsx
 ZIP File: ccd_sea_059_1516_w_1a_011717_xlsx.zip
     - ccd_sea_059_1516_w_1a_011717.xlsx
Year Folder: 17-18
 ZIP File: ccd_sea_059_1718_l_1a_083118.zip
     - ccd_sea_059_1718_l_1a_083118.csv
     - ccd_sea_059_1718_l_1a_083118.sas7bdat
 ZIP File: ccd_sea_052_1718_l_1a_083118.zip
     - ccd_sea_052_1718_l_1a_083118.csv
     - ccd_sea_052_1718_l_1a_083118.sas7bdat
Year Folder: 19-20
 ZIP File: ccd_sea_052_1920_l_1a_082120.zip
     - ccd_sea_052_1920_l_1a_082120.csv
     - ccd_sea_052_1920_l_1a_082120.sas7bdat
 ZIP File: ccd_sea_059_1920_l_1a_082120.zip
     - ccd_sea_059_1920_l_1a_082120.csv
     - ccd_sea_059_1920_l_1a_082120.sas7bdat
Year Folder: 21-22
 ZIP File: ccd_sea_05

In [5]:
# Extract CSV Files from ZIP and Load All Education Data
import pandas as pd
from pathlib import Path
import zipfile

def extract_year_from_folder(folder_name):
    """Convert '15-16' → 2015, '17-18' → 2017, etc."""
    start = int(folder_name.split('-')[0])
    return 2000 + start if start < 50 else 1900 + start

all_edu_rows = []

print("=== Loading Education Data ===")

for year_folder in sorted([f for f in EDU_DIR.iterdir() if f.is_dir()]):
    year_str = year_folder.name
    year = extract_year_from_folder(year_str)

    print(f"Processing {year_folder.name} → Year = {year}")

    # Process each zip
    for zfile in year_folder.iterdir():
        if zfile.suffix == ".zip":
            with zipfile.ZipFile(zfile, "r") as z:
                # Pick the CSV file inside the zip
                csv_files = [f for f in z.namelist() if f.endswith(".csv")]
                if not csv_files:
                    print("  ⚠️ No CSV found in:", zfile.name)
                    continue
                
                csv_name = csv_files[0]   # always 1 CSV per zip
                print(f"  ✔ Reading CSV: {csv_name}")

                with z.open(csv_name) as f:
                    df = pd.read_csv(f, low_memory=False)
                    df["year"] = year   # Add year column
                    df["source_file"] = zfile.name  # traceability

                    all_edu_rows.append(df)

print("\n=== Merge all years into one DataFrame ===")
edu_df = pd.concat(all_edu_rows, ignore_index=True)
print("Final Education Data Shape:", edu_df.shape)

edu_df.head()

=== Loading Education Data ===
Processing 15-16 → Year = 2015
  ✔ Reading CSV: ccd_sea_059_1516_w_1a_011717.csv
  ✔ Reading CSV: ccd_sea_052_1516_w_1a_011717.csv
  ⚠️ No CSV found in: ccd_sea_052_1516_w_1a_011717_xls.zip
  ⚠️ No CSV found in: ccd_sea_059_1516_w_1a_011717_xlsx.zip
Processing 17-18 → Year = 2017
  ✔ Reading CSV: ccd_sea_059_1718_l_1a_083118.csv
  ✔ Reading CSV: ccd_sea_052_1718_l_1a_083118.csv
Processing 19-20 → Year = 2019
  ✔ Reading CSV: ccd_sea_052_1920_l_1a_082120.csv
  ✔ Reading CSV: ccd_sea_059_1920_l_1a_082120.csv
Processing 21-22 → Year = 2021
  ✔ Reading CSV: ccd_sea_052_2122_l_1a_071722.csv
  ✔ Reading CSV: ccd_sea_059_2122_l_1a_071722.csv
Processing 23-24 → Year = 2023
  ✔ Reading CSV: ccd_sea_052_2324_l_1a_073124.csv
  ✔ Reading CSV: ccd_sea_059_2324_l_1a_073124.csv

=== Merge all years into one DataFrame ===
Final Education Data Shape: (62841, 357)


,SURVYEAR,FIPST,STABR,STATENAME,SEANAME,SCHSUP,SECTCH,STAFF,ELMGUI,KGTCH,...,ST,SEA_NAME,STATE_AGENCY_NO,STAFF_COUNT,TOTAL_INDICATOR,DMS_FLAG,GRADE,RACE_ETHNICITY,SEX,STUDENT_COUNT
0,2015-2016,1,AL,ALABAMA,Alabama Department Of Education,1993.04,19079.63,71628.43,928.05,4535.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-2016,2,AK,ALASKA,Alaska Department of Education and Early Devel...,1052.99,3507.40,16982.39,67.99,416.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-2016,4,AZ,ARIZONA,Arizona Department of Education,3790.96,14488.66,103174.6,448.17,2558.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-2016,5,AR,ARKANSAS,ARKANSAS DEPARTMENT OF EDUCATION,2926.65,14714.25,73658.2,577.88,2144.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-2016,6,CA,CALIFORNIA,California Department of Education,33873.81,84322.38,577836.1,2620.76,23014.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Education Data Profiling
print("=== Education Data Profiling ===\n")

print("Shape:", edu_df.shape)

# Preview column names
print("\n=== First 20 Columns ===")
print(list(edu_df.columns[:20]))

# Check survey years (string)
print("\n=== Unique SURVYEAR values ===")
print(edu_df['SURVYEAR'].unique())

# Check derived numeric year
print("\n=== Year Column Range ===")
print(edu_df['year'].min(), "→", edu_df['year'].max())

# Check states
print("\n=== Unique States (STABR) Count ===")
print(edu_df['STABR'].nunique())
print("\nSample STABR:", edu_df['STABR'].unique()[:15])

# Missing values summary
missing = edu_df.isna().sum().sort_values(ascending=False)
print("\n=== Missing Value Summary (Top 20) ===")
print(missing.head(20))

print("\n=== Percentage Missing (Top 20) ===")
print((missing / len(edu_df) * 100).head(20))

=== Education Data Profiling ===

Shape: (62841, 357)

=== First 20 Columns ===
['SURVYEAR', 'FIPST', 'STABR', 'STATENAME', 'SEANAME', 'SCHSUP', 'SECTCH', 'STAFF', 'ELMGUI', 'KGTCH', 'LEASUP', 'PARA', 'SECGUI', 'CORSUP', 'ELMTCH', 'LIBSUP', 'SCHADM', 'STUSUP', 'GUI', 'LEAADM']

=== Unique SURVYEAR values ===
['2015-2016' nan]

=== Year Column Range ===
2015 → 2023

=== Unique States (STABR) Count ===
57

Sample STABR: ['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN']

=== Missing Value Summary (Top 20) ===
TR06M    62784
AM10M    62784
WH10M    62784
BL10F    62784
BL10M    62784
HI10F    62784
HI10M    62784
AS10F    62784
AS10M    62784
AM10F    62784
TR09F    62784
AS12F    62784
TR09M    62784
HP09F    62784
HP09M    62784
WH09F    62784
WH09M    62784
BL09F    62784
BL09M    62784
HI09F    62784
dtype: int64

=== Percentage Missing (Top 20) ===
TR06M    99.909295
AM10M    99.909295
WH10M    99.909295
BL10F    99.909295
BL10M    99.909295
HI10F    99.909

### Education Data Overview

The education dataset contains **62,841 rows** and **357 columns**, covering multiple years of state-level K–12 education statistics from the CCD SEA files.

**Key fields include:**

- `SURVYEAR`: survey year label (most values are `"2015-2016"`)
- `year`: derived numeric year (ranges from **2015–2023**)
- `STABR`: 57 unique state abbreviations (includes states + D.C. + territories)
- `STATENAME`: full state name
- 300+ numerical indicators on staffing, enrollment, expenditures, and administration

### Missingness Summary

A large number of detailed demographic and finance variables have **extremely high missingness** (often > 99%).  
Most of these fields represent detailed subcategories only available for some states or years.

Common examples with heavy missingness include:

- `TR06M`, `AM10M`, `WH10M`, `BL10F`, `BL10M`, and many other subgroup variables
- Many columns exceed **99% missing** and do not contribute meaningful analytic value

In contrast, core variables such as `STABR`, `STATENAME`, and major staffing/enrollment totals have low or no missingness.

### State Coverage

The dataset includes **57 unique STABR values**, indicating:

- 50 U.S. states  
- Washington, D.C.  
- Additional territories (e.g., Puerto Rico, Guam, Virgin Islands)

Only states + D.C. will be used to match with crime data.

In [31]:
# === Step E3: Identify Usable Columns in Education Data ===

import pandas as pd
import numpy as np

print("=== Checking Column-Level Missing Percentages ===")

# Calculate % missing per column
edu_missing_pct = edu_df.isna().mean() * 100

# Sort descending
edu_missing_pct_sorted = edu_missing_pct.sort_values(ascending=False)

# Show top 30 columns with highest missing
print("\n=== Top 30 Columns with Highest Missing (%) ===")
print(edu_missing_pct_sorted.head(30))

# Show columns with < 5% missing (likely usable)
usable_cols = edu_missing_pct_sorted[edu_missing_pct_sorted < 5].index.tolist()

print("\n=== Columns With < 5% Missing (Likely Usable) ===")
print(usable_cols)

print(f"\nTotal usable columns (<5% missing): {len(usable_cols)}")

# Show columns with > 95% missing (likely discard)
discard_cols = edu_missing_pct_sorted[edu_missing_pct_sorted > 95].index.tolist()

print("\n=== Columns With > 95% Missing (Likely to Drop) ===")
print(discard_cols)
print(f"\nTotal columns >95% missing: {len(discard_cols)}")

=== Checking Column-Level Missing Percentages ===

=== Top 30 Columns with Highest Missing (%) ===
TR06M    99.909295
AM10M    99.909295
WH10M    99.909295
BL10F    99.909295
BL10M    99.909295
HI10F    99.909295
HI10M    99.909295
AS10F    99.909295
AS10M    99.909295
AM10F    99.909295
TR09F    99.909295
AS12F    99.909295
TR09M    99.909295
HP09F    99.909295
HP09M    99.909295
WH09F    99.909295
WH09M    99.909295
BL09F    99.909295
BL09M    99.909295
HI09F    99.909295
WH10F    99.909295
HP10M    99.909295
HP10F    99.909295
TR10M    99.909295
AM12F    99.909295
AM12M    99.909295
TR11F    99.909295
TR11M    99.909295
HP11F    99.909295
HP11M    99.909295
dtype: float64

=== Columns With < 5% Missing (Likely Usable) ===
['DMS_FLAG', 'TOTAL_INDICATOR', 'SEA_NAME', 'STATE_AGENCY_NO', 'ST', 'SCHOOL_YEAR', 'source_file', 'STATENAME', 'year', 'FIPST']

Total usable columns (<5% missing): 10

=== Columns With > 95% Missing (Likely to Drop) ===
['TR06M', 'AM10M', 'WH10M', 'BL10F', 'BL10M

### Education Data Profiling Summary

- The merged NCES CCD dataset contains **62,841 rows and 357 columns**.
- The majority of columns represent detailed race/grade/sex staff breakdowns.  
  However, **341 out of 357 columns (95%) have more than 95% missing values**, and thus are not suitable for analysis.
- Only a small subset of columns—including core state identifiers and aggregated staffing counts—have complete or near-complete coverage.
- The key usable identifier columns include:  
  `SURVYEAR`, `STABR`, `STATENAME`
- Many racial/grade-specific variables (e.g., `TR06M`, `AM10M`, `WH10F`, etc.) show **>99% missing** and will be removed during cleaning.
- The dataset spans survey years **2015–2023**, matching the project’s crime-data timeline.
- Unique states (STABR) ≈ **57**, consistent with US states + DC + territories.

Overall, the profiling indicates that the education dataset is highly sparse in detailed breakdown fields, and the cleaning step will focus on retaining only the core educational metrics.


### Education Data Cleaning Plan (Updated After Profiling)

1. **Filter to valid U.S. states only**  
   - The dataset includes 57 unique `STABR` values, which contain U.S. states, D.C., and additional territories (e.g., PR, VI, GU).  
   - Only the 50 states + D.C. will be kept to align with the crime dataset.

2. **Drop columns with extremely high missingness (>95%)**  
   - Profiling shows that **341 of 357 columns** exceed 95% missing, mostly detailed race × grade × gender staff counts.  
   - These variables are not consistently reported and will be removed.

3. **Retain only core, consistently available education indicators**  
   - Keep robust, aggregated variables such as:  
     - `STAFF` (total staff)  
     - `SECTCH` (secondary teachers)  
     - `ELMTCH` (elementary teachers)  
     - `SCHSUP` (school support staff)  
     - `STUSUP` (student support)  
     - `LEAADM` / `SCHADM` (administrative staff)  
     - Enrollment-related variables (e.g., `MEMBER`, `TOTAL`, `IMEMBER`)  
   - These fields are available across nearly all states and years.

4. **Ensure that the numeric `year` column is used for merging**  
   - The original `SURVYEAR` field contains strings like `"2015-2016"`.  
   - Profiling confirmed that the derived `year` column (2015–2023) is accurate and will be used for merging with crime data.

5. **Remove rows with missing or invalid state identifiers**  
   - Although most `STABR` values are valid, any rows with missing/invalid state codes will be dropped.

6. **Standardize column names and formats**  
   - Convert to lowercase for consistency.  
   - Rename key identifiers:  
     - `STABR` → `state`  
     - `STATENAME` → `state_name`

7. **Create additional derived indicators (optional but useful)**  
   - After cleaning, generate interpretable metrics:  
     - *Student–teacher ratio*  
     - *Teacher per 1,000 students*  
     - *Staff per 1,000 students*  
   - These will support clearer comparisons between states and years.

In [32]:
# Step 2 — Standardize column names: lowercase & replace spaces/hyphens
edu_clean = edu_df.copy()
edu_clean.columns = (
    edu_clean.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

print("Column name sample:", edu_clean.columns[:15])

Column name sample: Index(['survyear', 'fipst', 'stabr', 'statename', 'seaname', 'schsup',
       'sectch', 'staff', 'elmgui', 'kgtch', 'leasup', 'para', 'secgui',
       'corsup', 'elmtch'],
      dtype='object')


In [33]:
# Step 2 — Drop columns with >95% missing

missing_ratio = edu_clean.isna().mean()

drop_cols = missing_ratio[missing_ratio > 0.95].index.tolist()

print("Number of columns with >95% missing:", len(drop_cols))
print("Example of columns to drop:", drop_cols[:15])

edu_clean = edu_clean.drop(columns=drop_cols)

print("\nShape after dropping high-missing columns:", edu_clean.shape)

Number of columns with >95% missing: 341
Example of columns to drop: ['survyear', 'stabr', 'seaname', 'schsup', 'sectch', 'elmgui', 'kgtch', 'leasup', 'para', 'secgui', 'corsup', 'elmtch', 'libsup', 'schadm', 'stusup']

Shape after dropping high-missing columns: (62841, 16)


In [34]:
# Step 3 — Standardize column names

rename_map = {
    "FIPST": "state_fips",
    "STABR": "state_abbr",
    "STATENAME": "state",
}

edu_clean = edu_clean.rename(columns=rename_map)

edu_clean.columns = edu_clean.columns.str.lower()

edu_clean.head()

,fipst,statename,staff,year,source_file,school_year,st,sea_name,state_agency_no,staff_count,total_indicator,dms_flag,grade,race_ethnicity,sex,student_count
0,1,ALABAMA,71628.43,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ALASKA,16982.39,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,ARIZONA,103174.6,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,ARKANSAS,73658.2,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,CALIFORNIA,577836.1,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Step 4 — Clean year column & filter valid years

# Ensure year is numeric
edu_clean["year"] = pd.to_numeric(edu_clean["year"], errors="coerce")

print("Unique years before filtering:", sorted(edu_clean["year"].dropna().unique()))

# Keep only 2015–2023
edu_clean = edu_clean[(edu_clean["year"] >= 2015) & (edu_clean["year"] <= 2023)]

print("Shape after year filter:", edu_clean.shape)

edu_clean.head()

Unique years before filtering: [2015, 2017, 2019, 2021, 2023]
Shape after year filter: (62841, 16)


,fipst,statename,staff,year,source_file,school_year,st,sea_name,state_agency_no,staff_count,total_indicator,dms_flag,grade,race_ethnicity,sex,student_count
0,1,ALABAMA,71628.43,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ALASKA,16982.39,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,ARIZONA,103174.6,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,ARKANSAS,73658.2,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,CALIFORNIA,577836.1,2015,ccd_sea_059_1516_w_1a_011717_csv.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Step 5 — keep only useful columns & rename

keep_cols = ["fipst", "statename", "staff", "year"]
edu_clean = edu_clean[keep_cols].copy()

print("Shape after selecting core columns:", edu_clean.shape)
print("Columns now:", edu_clean.columns.tolist())

edu_clean = edu_clean.rename(
    columns={
        "statename": "state",
        "staff": "edu_staff_total",
        "fipst": "state_fips"
    }
)

before = edu_clean.shape[0]
edu_clean = edu_clean.dropna(subset=["state", "year", "edu_staff_total"])
after = edu_clean.shape[0]

print(f"Rows before dropping NA in key fields: {before}")
print(f"Rows after  dropping NA in key fields: {after}")
print(f"Removed: {before - after}")

edu_clean.head()

Shape after selecting core columns: (62841, 4)
Columns now: ['fipst', 'statename', 'staff', 'year']
Rows before dropping NA in key fields: 62841
Rows after  dropping NA in key fields: 5786
Removed: 57055


,state_fips,state,edu_staff_total,year
0,1,ALABAMA,71628.43,2015
1,2,ALASKA,16982.39,2015
2,4,ARIZONA,103174.6,2015
3,5,ARKANSAS,73658.2,2015
4,6,CALIFORNIA,577836.1,2015


In [37]:
from pathlib import Path
import os

BASE_DIR = Path(os.getcwd()).resolve().parent

clean_dir = BASE_DIR / "data" / "cleaned"
clean_dir.mkdir(parents=True, exist_ok=True)

output_file = clean_dir / "education_cleaned.csv"
edu_clean.to_csv(output_file, index=False)

# print("Saved cleaned education data to:", output_file)